In [ ]:
#For Colab

from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/MyDrive/CV_3
!ls

In [ ]:
!pip install -r requirements.txt

In [ ]:
import flask
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import os
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
import cv2
import numpy as np

from metrics import dice_loss, dice_coef, iou, dice_loss_m, dice_coef_m

In [ ]:
# put your token
!ngrok authtoken 26LPZYNETb2G2mIzWhe9ddHXkYv_6uvhfDYAfqsztiSy9JkZQ

In [ ]:
 # DeepLabV3 model opening
with CustomObjectScope({'iou': iou, 'dice_coef_m': dice_coef_m, 'dice_loss_m': dice_loss_m}):
    model = tf.keras.models.load_model("files/model_smooth=0.5.h5")

In [ ]:
# Defining prediction and saving function for app
def flask_predict(path):
  image = cv2.imread(path, cv2.IMREAD_COLOR)
  h, w, _ = image.shape
  x = cv2.resize(image, (512, 512))
  x = x/255.0
  x = x.astype(np.float32)
  x = np.expand_dims(x, axis=0)

  """ Prediction """
  y = model.predict(x)[0]
  y = cv2.resize(y, (w, h))
  y = np.expand_dims(y, axis=-1)

  """ Save the image """
  masked_image = image * y
  line = np.ones((h, 10, 3)) * 128
  cat_images = np.concatenate([image, line, masked_image], axis=1)
  cv2.imwrite(f"static/images/result.png", cat_images)

In [ ]:
#Defining app
image_folder = os.path.join('static','images/')
app = Flask(__name__)
run_with_ngrok(app) 
app.config["UPLOAD_FOLDER"] = image_folder

@app.route('/', methods=['GET'])
def home():
  return render_template('index.html')

@app.route('/', methods=['POST'])
def predict():

  imagefile = request.files['imagefile']
  image_path = image_folder + imagefile.filename

  # image will be saved in static/images/
  imagefile.save(image_path)

  pic = os.path.join('static/images', imagefile.filename)
  flask_predict(pic)
  return render_template('index.html', user_image="static/images/result.png", prediction_text='Result image and mask')

In [ ]:
# Run app and click on http://****.ngrok.io 
if __name__=='__main__':
  app.run()